In [1]:
import csv
import pandas as pd
from tqdm import tqdm
from requests.compat import urljoin
import requests
import glob
import os
import re
from bs4 import BeautifulSoup

In [2]:
# !python -m spacy download en_core_web_sm

In [3]:
# # !pip install spacy
# import spacy

# nlp = spacy.load("en_core_web_sm")

In [4]:
# Open types.txt and read the types
with open('../data/gold_standard_corpus.tsv', 'r') as f:
    pmcids = f.read().splitlines()

In [5]:
pmcids

['PMC4792959',
 'PMC4556948',
 'PMC5993813',
 'PMC3174205',
 'PMC5962829',
 'PMC3874094',
 'PMC3792120',
 'PMC4901335',
 'PMC3581133',
 'PMC2935479',
 'PMC5225553',
 'PMC5744400',
 'PMC3281816',
 'PMC3583137',
 'PMC4022742',
 'PMC3542345',
 'PMC4452330',
 'PMC4464872',
 'PMC4872455',
 'PMC3651197',
 'PMC3362782',
 'PMC5817132',
 'PMC4313693',
 'PMC4489904',
 'PMC4552872',
 'PMC5376652',
 'PMC5070310',
 'PMC5921292',
 'PMC5641157',
 'PMC3751948',
 'PMC5472290',
 'PMC4649626',
 'PMC5502978',
 'PMC4767726',
 'PMC3897916',
 'PMC5087830',
 'PMC3585192',
 'PMC5484670',
 'PMC5259676',
 'PMC3024232',
 'PMC3097211',
 'PMC5317055',
 'PMC3648400',
 'PMC5750880',
 'PMC5100220',
 'PMC4749753',
 'PMC5344356',
 'PMC5110973',
 'PMC5708618',
 'PMC3598673',
 'PMC3751959',
 'PMC5131611',
 'PMC5891595',
 'PMC3950279',
 'PMC5972578',
 'PMC5082793',
 'PMC5487420',
 'PMC5106849',
 'PMC3858553',
 'PMC3613406',
 'PMC4167147',
 'PMC3599585',
 'PMC2761781',
 'PMC3899050',
 'PMC2481430',
 'PMC5006041',
 'PMC47908

In [6]:
import requests
from urllib.parse import urljoin
import csv
from tqdm import tqdm

def get_Json_through_PMCID(pmcid, provider):
    base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
    article_url = urljoin(base_url, 
                          f"annotationsByArticleIds?articleIds=PMC%3A{pmcid}&provider={provider}&format=JSON")
    r = requests.get(article_url)

    if r.status_code == 200:
        return r
    else:
        return False

def get_epmc_annotations_to_file(PMCids):
    with open('../data/annotations_from_api.csv', 'w', newline='\n') as f1:
        test_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

        for each_id in tqdm(PMCids):
            # Loop through each provider
            for provider in ["Europe PMC", "PheneBank"]:
                json_annotations = get_Json_through_PMCID(each_id[3:], provider)  # Just the number is needed, so remove the PMC from the front
                if json_annotations:
                    json_results = json_annotations.json()
                    try:
                        pmc_id = json_results[0]['pmcid']
                        for each_annotation in json_results[0]['annotations']:
                            exact = each_annotation['prefix'] + each_annotation['exact'] + each_annotation['postfix']
                            token = each_annotation['tags'][0]['name']
                            ner = each_annotation['type']
                            row = [pmc_id, exact, token, ner, provider]
                            test_writer.writerow(row)
                    except(IndexError):
                        print('No annotations found for ' + str(each_id) + ' from ' + provider)
                else:
                    print('No annotations from ' + provider + ' for ' + str(each_id))
                    continue

# Usage example:
# PMCids = ['PMC123456', 'PMC654321', ...]
# get_epmc_annotations_to_file(PMCids)


In [7]:
# def get_Json_through_PMCID(pmcid):
#     base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
#     article_url = urljoin(base_url,
#                           "annotationsByArticleIds?articleIds=PMC%3A" + pmcid + "&provider=Europe%20PMC&format=JSON")
#     r = requests.get(article_url)

#     if r.status_code == 200:
#         return r
#     else:
#         return False


In [8]:
# def get_epmc_annotations_to_file(PMCids):
#     with open('../data/annotations_api.csv', 'w', newline='\n') as f1:
#         test_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

#         # count = 0
#         for each_id in tqdm(PMCids):
#             # count = count+1
#             # print(each_test_pmc_id + '\t' + str(count))
#             json_annotations = get_Json_through_PMCID(each_id[3:])  # Just the number is needed. So remove the PMC from the front
#             if json_annotations:
#                 json_results = json_annotations.json()
#                 try:
#                     pmc_id = json_results[0]['pmcid']
#                     # print(pmc_id)
#                     for each_annotation in json_results[0]['annotations']:
#                         exact = each_annotation['prefix'] + each_annotation['exact'] + each_annotation['postfix']
#                         token = each_annotation['tags'][0]['name']
#                         ner = each_annotation['type']
#                         row = [pmc_id, exact, token, ner]
#                         test_writer.writerow(row)
#                 except(IndexError):
#                     print('no annotations found!! '+str(each_id))
#             else:
#                 print('no annotations! '+str(each_id))
#                 continue


In [9]:
# get_epmc_annotations_to_file(pmcids)

In [10]:
annotations_df = pd.read_csv('../data/annotations_from_api.csv', sep='\t', names=['pmcid', 'partial_sentence', 'token', 'ner', 'provider'])
annotations_df

,pmcid,partial_sentence,token,ner,provider
0,PMC4792959,A subunit of the oligosaccharyltransferase com...,oligosaccharyltransferase complex,Gene Ontology,Europe PMC
1,PMC4792959,"During plant pollen tube (PT) reception, gamet...",pollen tube,Gene Ontology,Europe PMC
2,PMC4792959,"ing in PT rupture, sperm release, and double f...",double fertilization,Gene Ontology,Europe PMC
3,PMC4792959,fic crosses between Arabidopsis thaliana and r...,Arabidopsis thaliana,Organisms,Europe PMC
4,PMC4792959,T3/6 subunit of the oligosaccharyltransferase ...,oligosaccharyltransferase complex,Gene Ontology,Europe PMC
...,...,...,...,...,...
89254,PMC3458065,"plasma viral load, p24-antigen and HIV cap",p24,Gene_Proteins,Europe PMC
89255,PMC3458065,"ad, p24-antigen and HIV capture by erythroc",HIV,Organisms,Europe PMC
89256,PMC3458065,by erythrocytes in HIV-positive individual,HIV,Organisms,Europe PMC
89257,PMC3458065,association between detectable plasma viral lo...,plasma,Anatomy,PheneBank


In [11]:
annotations_df['ner'].unique()

array(['Gene Ontology', 'Organisms', 'Gene_Proteins',
       'Experimental Methods', 'Diseases', 'Chemicals',
       'Accession Numbers', 'Resources', 'Molecule', 'Phenotype',
       'Anatomy', 'Cell', 'Pathway', 'Gene Mutations'], dtype=object)

In [12]:
annotations_df_other = annotations_df[annotations_df['ner'].isin(['Gene Ontology', 'Experimental Methods', 'Accession Numbers', 'Resources', 'Chemicals'])]
annotations_df_other

,pmcid,partial_sentence,token,ner,provider
0,PMC4792959,A subunit of the oligosaccharyltransferase com...,oligosaccharyltransferase complex,Gene Ontology,Europe PMC
1,PMC4792959,"During plant pollen tube (PT) reception, gamet...",pollen tube,Gene Ontology,Europe PMC
2,PMC4792959,"ing in PT rupture, sperm release, and double f...",double fertilization,Gene Ontology,Europe PMC
4,PMC4792959,T3/6 subunit of the oligosaccharyltransferase ...,oligosaccharyltransferase complex,Gene Ontology,Europe PMC
6,PMC4792959,nsferase complex conferring protein N-glycosyl...,glycosylation,Experimental Methods,Europe PMC
...,...,...,...,...,...
89201,PMC3458065,The viral capture assay was performed after t,assay,Experimental Methods,Europe PMC
89220,PMC3458065,ated for 7 to 10 days and assayed for syncytiu...,syncytium formation,Gene Ontology,Europe PMC
89221,PMC3458065,is able to inhibit syncytium formation (loss ...,syncytium formation,Gene Ontology,Europe PMC
89228,PMC3458065,end-point dilution assay in MT-2 cells cultu,assay,Experimental Methods,Europe PMC


In [13]:
# # Get unique PMCIDs
# unique_pmcids = annotations_df_other['pmcid'].unique()url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML"
# unique_pmcids

# import requests
# from bs4 import BeautifulSoup
# import re

# def get_full_text_xml(pmcid):
#     url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML"
#     response = requests.get(url)
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, 'xml')
#         p_tags = soup.find_all('p')
#         p_texts = [tag.get_text() for tag in p_tags]
#         return p_texts
#     else:
#         return None

In [14]:
unique_pmcids = annotations_df_other['pmcid'].unique()

In [15]:
def get_full_text_xml(pmcid):
    path = f"../data/300_articles_source_files/{pmcid}.xml"
    try:
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        soup = BeautifulSoup(content, 'lxml-xml')  # Use lxml-xml parser
        plain_tags = soup.find_all('plain')
        plain_texts = [tag.get_text() for tag in plain_tags]
        return plain_texts
    except FileNotFoundError:
        print(f"File {pmcid}.xml not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [24]:
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

def check_previous_word_pos(sentence, target_word):
    # Process the sentence to get tokenized words and their POS tags
    doc = nlp(sentence)
    
    # Find the target word and get the POS of the previous word
    for i, token in enumerate(doc):
        if token.text == target_word and i > 0:
            previous_word = doc[i - 1]
            pos_of_previous = previous_word.pos_
            
            # Check if the POS is one of the ones we want to avoid
            if pos_of_previous in ['DET', 'ADV', 'VERB', 'ADP']:
                return False  # POS to be avoided
            else:
                return True  # Acceptable POS

    return False  # Return False if the target word is not found or is the first word


In [85]:
def find_sentence_with_substring(string_list, substring):
    for text in string_list:
        sentences = re.split(r'(?<=[.!?])\s+', text)
        for sentence in sentences:
            if substring in sentence:
                return sentence
    return None

def check_previous_word_pos(sentence, prev_word):
    # Process the sentence to get tokenized words and their POS tags
    doc = nlp(sentence)
    
    for token in doc:
        if token.text == prev_word:
            if token.i > 0:
                pos_of_previous = doc[token.i - 1].pos_
                return pos_of_previous not in ['DET', 'ADV', 'VERB', 'ADP', 'NUM', 'CONJ', 'SCONJ']

    return False

def adjust_token_for_context(sentence, token, ner_):
    check_list = ['the', 'an', 'a', 'that', 'after', 'and', 'total', 'using', 'by','as', 'for', 'in', 'with', 
                  'underwent', 'of', 'with', 'but', 'or', 'before', 'at', 
                  'activate', 'research', 'favors', 'prevent', 'to', 'drive', 'on', 'both', 'performed', 'upon',
                 'calculate', 'like', 'μl', 'support', 'our', 'core', 'eight', 'was', 
                  'nested', 'show', 'designed', '3', 'each', 'uts', 'these', 'when' '1', 'no', 
                  'from', 'if', 'individual', 'beyond', 'used', 'facilitated', 
                  'when', 'perform', 'involving', 'time', 'were', 'rich', '1x', 'this', 
                  'Briefly,', 'generated', 'following', 'since']
    
    specific_phrases = ['viral capture assay', 'flow cytometry assay', 
                        'luminescent cell viability assay', 'real-time PCR', 'RT-PCR', 'T-DNA insertion']
    
    
    words = sentence.split(' ')
    punctuation_marks = [',', '.', '?', '!', ';', ']','[',')','(', ':']
    extended_tokens = set()
    
   
    for phrase in specific_phrases:
        # Find all case-insensitive occurrences of the phrase
        found_phrases = re.findall(phrase, sentence, re.IGNORECASE)
        # Add each found phrase to the extended tokens
        for found in found_phrases:
            extended_tokens.add(found)
            
    # Remove parentheses from the token
    token_cleaned = token.replace('(', '').replace(')', '')

    # Adjust token for larger term if token is a substring and does not contain specific punctuation
    if ner_ not in['Chemicals', 'Accession Numbers']:
        for word in words:
            cleaned_word = word.replace('(', '').replace(')', '')
            if token_cleaned in cleaned_word and cleaned_word != token_cleaned and not any(mark in cleaned_word for mark in punctuation_marks):
                extended_tokens.add(cleaned_word)
            else:
                extended_tokens.add(token_cleaned)
    else:
        extended_tokens.add(token_cleaned)
            
    

    if ner_ in ['Experimental Methods']:
    # Check for words immediately before the token and add those combinations
        for i, word in enumerate(words):
            cleaned_word = word.replace('(', '').replace(')', '')
            if cleaned_word == token_cleaned and i > 0:
                prev_word = words[i - 1].replace('(', '').replace(')', '')
                if token_cleaned.lower() in ['assay', 'assays', 'insertion', 'insertions', 'elisa', 'pcr', 
                                             'mda', 'iss' 'biopsy', 'co-expression', 
                                             'co-expressions', ' transfection']:
                    if prev_word.lower() not in check_list and (prev_word.isupper() or not prev_word.islower()):
                        extended_tokens.add(prev_word + ' ' + token_cleaned)
                elif prev_word.lower() not in check_list and check_previous_word_pos(sentence, prev_word):
                    extended_tokens.add(prev_word + ' ' + token_cleaned)

        # Always add the token itself, after removing parentheses
        extended_tokens.add(token_cleaned)

    return list(extended_tokens)




def process_pmcid(df, pmcid, p_texts):
    sentences_data = {}
    for _, row in df[df['pmcid'] == pmcid].iterrows():
        sentence = find_sentence_with_substring(p_texts, row['partial_sentence'])
        if sentence:
            if sentence not in sentences_data:
                sentences_data[sentence] = set()

            updated_tokens = adjust_token_for_context(sentence, row['token'], row['ner'])
            for updated_token in updated_tokens:
                sentences_data[sentence].add((updated_token, row['ner']))

    return [[pmcid, sentence, list(ner_tags)] for sentence, ner_tags in sentences_data.items()]

final_data = []

for pmcid in tqdm(unique_pmcids):
    p_texts = get_full_text_xml(pmcid)
    if p_texts:
        processed_data = process_pmcid(annotations_df_other, pmcid, p_texts)
        final_data.extend(processed_data)

# Convert to DataFrame
final_df = pd.DataFrame(final_data, columns=['pmcid', 'sentence', 'ner'])

# Save as TSV
final_df.to_csv('../data/xxx7.tsv', sep='\t', index=False)

100%|█████████████████████████████████████████| 299/299 [00:51<00:00,  5.79it/s]


In [86]:
import ast

def calculate_spans(sentence, entities):
    spans = []
    for entity, etype in entities:
        start = sentence.find(entity)
        if start != -1:
            end = start + len(entity)
            spans.append([start, end, entity, etype])  # List instead of tuple
    return spans

def resolve_overlaps(spans):
    spans.sort(key=lambda x: (x[1] - x[0]), reverse=True)  # Sort by span length
    resolved = []
    for span in spans:
        overlap = False
        for r in resolved:
            if not (span[1] <= r[0] or span[0] >= r[1]):  # Check for overlap
                overlap = True
                break
        if not overlap:
            resolved.append(span)
    return resolved



In [87]:
df = pd.read_csv('../data/xxx7.tsv', sep = '\t')
df['new_ner'] = df.apply(lambda row: resolve_overlaps(calculate_spans(row['sentence'], ast.literal_eval(row['ner']))), axis=1)
df.drop('ner', axis=1, inplace=True)
df.rename(columns={'new_ner': 'ner'}, inplace=True)
df.to_csv('../data/xxx6.tsv', sep = '\t', index=False)

In [88]:
# Rename entity names in file2.csv as per the provided mapping
entity_rename_map = {
    'Gene Ontology': 'GO',
    'Experimental Methods': 'EM', 
    'Accession Numbers': 'AN',
    'Resources': 'RS',
    'Chemicals': 'CD'
}

file2_df = pd.read_csv('../data/xxx6.tsv', sep = '\t')
# Applying the renaming
file2_df['ner'] = file2_df['ner'].apply(lambda x: eval(x))
for row in file2_df['ner']:
    for entity in row:
        entity[3] = entity_rename_map.get(entity[3], entity[3])

# Checking the changes
file2_df.head()


,pmcid,sentence,ner
0,PMC4792959,A subunit of the oligosaccharyltransferase com...,"[[17, 50, oligosaccharyltransferase complex, GO]]"
1,PMC4792959,"During plant pollen tube (PT) reception, gamet...","[[13, 24, pollen tube, GO]]"
2,PMC4792959,ARU encodes the OST3/6 subunit of the oligosac...,"[[38, 71, oligosaccharyltransferase complex, GO]]"
3,PMC4792959,Our results suggest that glycosylation pattern...,"[[25, 38, glycosylation, EM]]"
4,PMC4792959,show that ARTUMES regulates pollen tube recogn...,"[[28, 39, pollen tube, GO]]"


In [105]:
# Correcting the code to work with the 'ners' column, use sets to remove duplicates, 
# and applying specific handling for 'AN' type

# Adjusting the safe_eval_list function to check if the input is a string
def safe_eval_list(str_list):
    if isinstance(str_list, str):
        try:
            return eval(str_list)
        except SyntaxError:
            return []
    return str_list

# Applying the adjusted renaming and extraction
file2_df['ner'] = file2_df['ner'].apply(safe_eval_list)
for row in file2_df['ner']:
    for entity in row:
        entity[3] = entity_rename_map.get(entity[3], entity[3])

        
def replace_substrings(input_str):
    substrings_to_replace = ['*/', '18O ', '% ', 'μm', 'its ', 'are ',  'successful ', 'III ', '6 ', 
                             'their ', 'μm ', 'between ', 'some ', 'through ', 'mL ', 'while ','s layer', '50 ', 
                             'vs ', 'induces ', 'p53 ', 'promotes ', 'within ', 'precise ', 'nitrogen ', '—0', 
                             'other ', 'created ', 'basement ', 'best ', 'study ', 'than ', '94°C,', 'followed ', 
                             '°CAbbreviations:', '7500', 'simultaneous', 'represent', 'II ', '15 ', '10× ', 
                             'initiation ', 'all ', 'toward ', 'improve ', 'whose ', 
                             'influence ', 'convey', 'and/or ', 'acid ', 
                             'analyzed ', 'thus ', 'per ', 'affect ', 'associated ', 'increased' , 
                             'releases ', 'kill', 'Nonetheless, ', 
                            '“', 'against ', 'additional ', 'late ', 'reduced ', 'Distribution ', 
'giving ', 'maintain ', 'nighttime ', 'increase ', 'during ', '30 ' 'HIV ',
'whether ', '- ', 'use ', 'Three ', 'Caused', 'block ', 'IL-8 ', 'No ', 'create ', '”'
'Il-21 ', 'what ', 'base ', 'paid ', 'then ', 'without ', 'where ', '16100 ', '2 ', '325,000 ', 'underlying ', 'related ']
    
    for substring in substrings_to_replace:
        input_str = input_str.replace(substring, '')
    
    return input_str


# Extracting dictionary collection of set with AN, EM, RS, and GO
extracted_entities = {'AN': set(), 'EM': set(), 'RS': set(), 'GO': set(), 'CD': set()}
for row in file2_df['ner']:
    for entity in row:
        entity_type = entity[3]
        if entity_type in extracted_entities:
            # For 'AN', split with a space and take the last term
            if entity_type == 'AN':
                extracted_entities[entity_type].add(entity[2].split()[-1])
            elif entity_type == 'RS' and 'database' in entity[2]:
                extracted_entities[entity_type].add(entity[2].replace('database','').strip())
            elif entity_type == 'GO' or entity_type == 'EM':
                if 'membrane' in entity[2].lower() or 'behaviour' in entity[2].lower():
                    extracted_entities[entity_type].add(entity[2].split()[-1])
                else:
                    replac_str = replace_substrings(entity[2]).strip()
                    if replac_str and len(replac_str)>1:
                        extracted_entities[entity_type].add(replac_str)
            else:
                extracted_entities[entity_type].add(entity[2])


# Converting sets back to lists for easier viewing/usage
extracted_entities = {k: list(v) for k, v in extracted_entities.items()}

# extracted_entities



In [106]:
extracted_entities['RS'].extend(['NHGRI', 'GEO', 'Sequence Read Archive', 'SRA', 'KEGG', 
                                 'Kyoto Encyclopedia of Genes and Genomes', 'IntAct62',
                                 'MINT63', 'AtPIN36', 'AtPID37', 'PAIR38', 'APID39', 'PRIN9', 'PHISTO', 'TAIR',
                                'InParanoid', 'HPRD', 'Medline', 'EMBASE', 'PsycINFO', 'EconLit', 'HEED',
                                 'CRD', 'Health Economic Evaluations Database', 'HEED', 'NHS Economic Evaluation Database',
                                'NHS EED', 'Database of Abstracts of Reviews of Effects', 'DARE', 'PROSITE',
                                'CINAHL', 'Cochrane', 'ARAMEMNON', 'TrEMBL', 'PhosphoSitePlus', 
                                 'UniProtKB/TrEMBL', 'SwissProt', 'miRWalk',
                                 'DAVID', 'SEER', 'Genebank','MSigDB', 'MGI', 'PsyGeNET', 'SFARI', 'DisGeNET',
                                'GeoSymbio', 'TIGR', 'GenBank' ])

In [107]:
extracted_entities['AN']

['rs3793275',
 'rs2970836',
 'NCT01079247',
 'GSM1334019',
 'rs3752823',
 'GCA_000471725.1',
 'AAK01938',
 'HM045017',
 '4m94',
 '2fvp',
 'GO:0045595',
 'AGV04659',
 'SRP006767',
 '123910',
 'Y07682',
 'Q13151',
 'AAF74025',
 'NCT03200795',
 'HM020130',
 'P25297',
 'MF468328',
 'EV253998',
 'GSM1334015',
 'rs4716990',
 'A00149',
 'O48639',
 'SRP002176',
 '4LY1',
 '606766',
 'P55209',
 '3JWN',
 'Q6WRI0',
 '153700',
 'GSE89919',
 'GSM1334017',
 'NCT03022370',
 'HQ223450',
 'ENSGALG00000005155',
 'EF613347',
 'GSE63382',
 'Q43650',
 'SRR4011621',
 'rs2961365',
 'Q9H254',
 'rs6836703',
 '2J3S',
 'Q00908',
 'SRR4011626',
 'Q13637',
 '4IGG',
 'HM020128',
 'GCA_000247795.2',
 '2KXA',
 'Q12873',
 'rs316762',
 'Q5S007',
 'KR269920',
 'KR269940',
 'Q5F3J4',
 'SRP002459',
 '2PRG',
 '2BRQ',
 'Q9NQC3',
 'O60641',
 'MH238460',
 '1ztw',
 'AAL37552',
 'HM019533',
 'AAK38197',
 'Q4V958',
 'rs4986791',
 '122920',
 'ENST00000355810',
 'P07205',
 'rs10512793',
 'KY055664',
 '1K47',
 'Y07681',
 'AW310136',

In [108]:
extracted_entities['AN'].extend(['GSE50635','MF468290', 'SRR4011627', 'SRR4011628'])

In [109]:
extracted_entities['GO'].remove('part')
extracted_entities['RS'].remove('HPA')

In [110]:
import pandas as pd

def enrich_entities(file_path, new_entity_dict):
    # Load the CSV file
    df = pd.read_csv(file_path, names=['sentence', 'ner'])

    # Function to find if there is an overlap with existing spans or if the term is a substring
    def is_valid_entity(sentence, new_span, existing_spans, current_entity_type):
        start_idx, end_idx = new_span
        # Check for overlap with existing spans and allow 'AN' entities to override 'CD'
        for span, span_type in existing_spans:
            if not (end_idx <= span[0] or start_idx >= span[1]):
                # Allow 'AN' to override 'CD'
                if current_entity_type == 'AN' and span_type == 'CD':
                    continue
                return False
        # Check if the entity is not part of a larger word (to avoid substrings)
        if (start_idx > 0 and sentence[start_idx - 1].isalnum()) or (end_idx < len(sentence) and sentence[end_idx].isalnum()):
            return False
        return True

    # Function to add new entity annotations
    def add_new_entities(sentence, entities, entity_dict):
        existing_spans = [(tuple(map(int, entity[:2])), entity[3]) for entity in entities if entity != 'None']  # Tuple of span and type
        for entity_type, terms in entity_dict.items():
            for term in terms:
                start_idx = sentence.find(term)
                while start_idx != -1:
                    end_idx = start_idx + len(term)
                    new_span = (start_idx, end_idx)
                    if is_valid_entity(sentence, new_span, existing_spans, entity_type):
                        entities = [entity for entity in entities if not (entity[0] == start_idx and entity[3] == 'CD')]  # Remove overridden 'CD'
                        entities.append([start_idx, end_idx, term, entity_type])
                        existing_spans.append((new_span, entity_type))
                    start_idx = sentence.find(term, start_idx + 1)
        return [entity for entity in entities if entity != 'None']  # Exclude 'None' if other entities are present

    # Enriching the entity annotations
    for i, row in df.iterrows():
        sentence = row['sentence']
        entities = eval(row['ner']) if row['ner'] != "['None']" else ['None']
        df.at[i, 'ner'] = add_new_entities(sentence, entities, new_entity_dict)

    return df



In [102]:
def find_sub_span(sub_span_range, full_spans_range):
    # Check if a sub span is present in full span
    if sub_span_range[0] in range(full_spans_range[0], full_spans_range[1]):
        return sub_span_range

def convert2IOB(text_data, ner_tags):
    if 'None' in ner_tags:
        # Split text into words and return with 'O' tags if there are no NER tags
        split_text = tokenizer.tokenize(text_data)
        return zip(split_text, ['O'] * len(split_text))

    # Tokenize text and create a list of token spans
    tokens = tokenizer.tokenize(text_data)
    current_pos = 0
    token_spans = []
    for token in tokens:
        start_pos = text_data.find(token, current_pos)
        end_pos = start_pos + len(token)
        token_spans.append((start_pos, end_pos))
        current_pos = end_pos

    # Initialize IOB tags as 'O'
    iob_tags = ['O'] * len(tokens)

    # Process each NER tag
    for start, end, entity, entity_type in sorted(ner_tags, key=lambda x: x[0]):
        for i, span in enumerate(token_spans):
            if start <= span[0] < end:
                if start == span[0]:
                    # Beginning of entity
                    iob_tags[i] = 'B-' + entity_type
                else:
                    # Inside of entity
                    iob_tags[i] = 'I-' + entity_type

    return zip(tokens, iob_tags)



In [103]:
import csv
import pathlib
from nltk.tokenize import WordPunctTokenizer, wordpunct_tokenize
tokenizer = WordPunctTokenizer()

def convert_to_IOB_format(data_file, output_folder,filename):
    # Read the CSV file
    df = pd.read_csv(data_file)

    # Define the output path
    result_path = os.path.join(output_folder, filename)
    pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    with open(result_path, 'w', newline='\n') as f1:
        train_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
        
        for index, row in df.iterrows():
            text = row['sentence']
            ner_tags = eval(row['ner']) if row['ner'] != "['None']" else ['None']
            tagged_tokens = convert2IOB(text, ner_tags)
            for each_token in tagged_tokens:
                train_writer.writerow(list(each_token))
            train_writer.writerow('')

output_folder = '../data/enriched_IOB/'

In [116]:
# Example usage
filename = 'dev.csv'

file_path = '../data/CD_GP_DS_OG_test/clean_CD_GP_DS_OG_'+filename
new_df = enrich_entities(file_path, extracted_entities)
new_df.to_csv(file_path.replace('clean_','enriched_'), index=False)

In [117]:
data_file = '../data/CD_GP_DS_OG_test/enriched_CD_GP_DS_OG_'+filename
convert_to_IOB_format(data_file, output_folder,filename)